In [1]:
## import necessary packages 
## we are using pyomo for optimization

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from io import StringIO
from scipy.stats import norm

from pylab import *
from pyomo.environ import * 

### Product Mix

#### Manually

In [5]:
model = ConcreteModel("Product Mix - Solved Manually") ##the open parenthesis allow us to later name the model by putting the name in single quotes in the parenthesis

#step 1: identify the decision variables 
#DVS
#model.something will make up all
#defining the DVs as variables 
#other options for domain are binary, integer (which means a whole number but also negative or zero), nonnegative reals
model.r = Var(domain=NonNegativeIntegers) #model.r is the name for the DV roses 
model.b = Var(domain=NonNegativeIntegers)


#step 2: define the objective function
#Obj. Fct.
#Objective() tells python this is the objective function
#expr stands for expression
#sense= allows up to tell python what to do. the only two we use is maximize or minimize
model.profit = Objective(expr = 2.25*model.r + 2.6*model.b, sense=maximize)


#step 3: identify the constraints 
#Contraint() tells python this is the objective function
#equality options: <=, >=, ==
model.f1 = Constraint(expr = 2*model.r + model.b <=4000)
model.f2 = Constraint(expr = model.r + 2*model.b <=5000)

SolverFactory('glpk').solve(model)

model.display()

Model Product Mix - Solved Manually

  Variables:
    r : Size=1, Index=None
        Key  : Lower : Value  : Upper : Fixed : Stale : Domain
        None :     0 : 1000.0 :  None : False : False : NonNegativeIntegers
    b : Size=1, Index=None
        Key  : Lower : Value  : Upper : Fixed : Stale : Domain
        None :     0 : 2000.0 :  None : False : False : NonNegativeIntegers

  Objectives:
    profit : Size=1, Index=None, Active=True
        Key  : Active : Value
        None :   True : 7450.0

  Constraints:
    f1 : Size=1
        Key  : Lower : Body   : Upper
        None :  None : 4000.0 : 4000.0
    f2 : Size=1
        Key  : Lower : Body   : Upper
        None :  None : 5000.0 : 5000.0


In [4]:
print(f'''
Profit:    {model.profit()}
R to plant: {model.r()}
B to plant: {model.b()}
F1: {model.f1()}
F2: {model.f2()}
''')


Profit:    7450.0
R to plant: 1000.0
B to plant: 2000.0
F1: 4000.0
F2: 5000.0



#### Pyomo Method

In [8]:
profit = pd.read_excel("Portfolio3Data.xlsx",index_col=0)
data = pd.read_excel("Portfolio3Data.xlsx", sheet_name="PMData", index_col=0)

In [70]:
profit

,Roses,Begonias
Profit,2.25,2.6


In [71]:
data

,Roses,Begonias,Available
F1,2,1,4000
F2,1,2,5000


In [ ]:
## we now have the data, but we need to put in the objective function, DVs, and constraints

In [ ]:
##index set: list of individual headers. is a list that we can iterate over
##in our data we have two index sets: 
# 1. roses and begonias. if we were to extract the index set for Roses and Begonias,
#we get it from table #1 so we dont have to chop off "available"
#always extract DV index set from obj func table
# 2. F1 and F2. if we were to extract the index set for F1 and F2, our only option is to get it from table #2

In [9]:
model = ConcreteModel("Product Mix - Solved with Pyomo Method")

P = profit.keys() ##keys in a dataframe are the column headers
#the set 'P' has two elements, which are our DVs

##DECISION VARIABLES
model.x = Var(P, domain=NonNegativeReals)
#telling python to declare variables across the set P 
#we are declaring both DVs at the same time

##OBJECTIVE FUNCTION
#now that we have our DVs we can articulate the obj func
model.profit = Objective(expr = sum([model.x[p]*profit.loc['Profit',p]for p in P]), sense = maximize)
                                     
# we sum model.x subscipt p * the profit row elements
#when you give python a location, you specify the location by row, column. so we're in profit row and column p. right now p doesn't mean anything until we say for p in P
#P is the index set consisting of roses and begonias.
#the line of command is doing a sumproduct 

##CONSTRAINTS
I=data.index
#data is the constraint table we read in
#the index command gives us the row headers: F1, F2

##create empty constraint list
model.cons = ConstraintList()

for i in I:
    model.cons.add(sum([model.x[p]*data.loc[i,p] for p in P]) <= data.loc[i, 'Available'])
    
    
## the .add() function allows us to add constraints
##in a series of nested loops, the first "for i in I" is the outer loop and the second "for i in I" is the inner loop
##use the outer loop for each constraint. aka: for each outer loop, create a new constraint
##we create constraint list and then add all constraints to it
#index = row headers
#keys = column headers

##once python starts the loop, it needs to finish the inner loop for a given outer loop before it starts on the next outer loop
## starts with F1 (for i in I, we start with F1) and apply functions to roses, and then begonias, and then start over entire loop for F2                                                                        
                                                                          
SolverFactory('glpk').solve(model)
model.display()

Model Product Mix - Solved with Pyomo Method

  Variables:
    x : Size=2, Index={Begonias, Roses}
        Key      : Lower : Value  : Upper : Fixed : Stale : Domain
        Begonias :     0 : 2000.0 :  None : False : False : NonNegativeReals
           Roses :     0 : 1000.0 :  None : False : False : NonNegativeReals

  Objectives:
    profit : Size=1, Index=None, Active=True
        Key  : Active : Value
        None :   True : 7450.0

  Constraints:
    cons : Size=2
        Key : Lower : Body   : Upper
          1 :  None : 4000.0 : 4000.0
          2 :  None : 5000.0 : 5000.0


### Product Mix Continued

If your fertilizer storage capacity is 8000 pounds, what is your new product mix, and how much of each fertilizer should you order?

What would you be willing to pay for additional storage capacity?

Now how much do you plant?

In [17]:
model = ConcreteModel("Product Mix Continued - Solved Manually") ##the open parenthesis allow us to later name the model by putting the name in single quotes in the parenthesis

#step 1: identify the decision variables 
model.r = Var(domain=NonNegativeIntegers) # no. roses to plant
model.b = Var(domain=NonNegativeIntegers) # no. begonias to plant


#step 2: define the objective function
#Obj. Fct.
#Objective() tells python this is the objective function
#expr stands for expression
#sense= allows up to tell python what to do. the only two we use is maximize or minimize
model.profit = Objective(expr = 2.25*model.r + 2.6*model.b, sense=maximize)


#step 3: identify the constraints 
#Contraint() tells python this is the objective function
#equality options: <=, >=, ==
model.f1 = Constraint(expr = 2*model.r + model.b <=4000) #f1 capacity
model.f2 = Constraint(expr = model.r + 2*model.b <=5000) #f2 capacity

model.storage= Constraint(expr = 3*model.r + 3*model.b <= 8000) #total storage capacity

SolverFactory('glpk').solve(model)
model.display()

Model Product Mix Continued - Solved Manually

  Variables:
    r : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 : 332.0 :  None : False : False : NonNegativeIntegers
    b : Size=1, Index=None
        Key  : Lower : Value  : Upper : Fixed : Stale : Domain
        None :     0 : 2334.0 :  None : False : False : NonNegativeIntegers

  Objectives:
    profit : Size=1, Index=None, Active=True
        Key  : Active : Value
        None :   True : 6815.400000000001

  Constraints:
    f1 : Size=1
        Key  : Lower : Body   : Upper
        None :  None : 2998.0 : 4000.0
    f2 : Size=1
        Key  : Lower : Body   : Upper
        None :  None : 5000.0 : 5000.0
    storage : Size=1
        Key  : Lower : Body   : Upper
        None :  None : 7998.0 : 8000.0


### Dual Problem

In [2]:
## solve manually

In [5]:
model = ConcreteModel("Dual Problem - Solved Manually") ##the open parenthesis allow us to later name the model by putting the name in single quotes in the parenthesis

#step 1: identify the decision variables 
#DVS
model.fertilizerone = Var(domain=NonNegativeIntegers) #F1 shadow price
model.fertilizertwo = Var(domain=NonNegativeIntegers) #F2 shadow price
model.storage = Var(domain=NonNegativeIntegers) #total storage shadow price 


#step 2: define the objective function
#Obj. Fct.
#Objective() tells python this is the objective function
#expr stands for expression
#sense= allows up to tell python what to do. the only two we use is maximize or minimize
model.profit = Objective(expr = 4000*model.fertilizerone + 5000*model.fertilizertwo + 8000*model.storage, sense=minimize)


#step 3: identify the constraints 
#Contraint() tells python this is the objective function
#equality options: <=, >=, ==

model.roses = Constraint(expr = 2*model.fertilizerone + 1*model.fertilizertwo + 3*model.storage >= 2.25) #roses constraint
model.begonias = Constraint(expr = 1*model.fertilizerone + 2*model.fertilizertwo + 3*model.storage >= 2.6) #begonias constraint

SolverFactory('glpk').solve(model)
model.display()

Model Dual Problem - Solved Manually

  Variables:
    fertilizerone : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :   0.0 :  None : False : False : NonNegativeIntegers
    fertilizertwo : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :   0.0 :  None : False : False : NonNegativeIntegers
    storage : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :   1.0 :  None : False : False : NonNegativeIntegers

  Objectives:
    profit : Size=1, Index=None, Active=True
        Key  : Active : Value
        None :   True : 8000.0

  Constraints:
    roses : Size=1
        Key  : Lower : Body : Upper
        None :  2.25 :  3.0 :  None
    begonias : Size=1
        Key  : Lower : Body : Upper
        None :   2.6 :  3.0 :  None


In [3]:
## solve with pyomo

In [13]:
DualProblemPrice = pd.read_excel("Portfolio3Data.xlsx", sheet_name="DualProblemPrice", index_col=0)
DualProblemData = pd.read_excel("Portfolio3Data.xlsx", sheet_name="DualProblemData", index_col=0)

In [81]:
DualProblemPrice

,Fertilizer One,Fertilizer Two,Storage
Price,4000,5000,8000


In [82]:
DualProblemData

,Fertilizer One,Fertilizer Two,Storage,Profit
Rose,2,1,3,2.25
Begonia,1,2,3,2.60


In [14]:
model = ConcreteModel("Dual Problem - Solved with Pyomo Method")

D = DualProblemPrice.keys()

model.dv_dualproblem = Var(D, domain = NonNegativeIntegers)

model.cost = Objective(expr = 
                       sum([
                           model.dv_dualproblem[d] * 
                           DualProblemPrice.loc['Price',d]
                           for d in D
                       ]),
                       sense=minimize)

model.cons = ConstraintList()

F = DualProblemData.index

for f in F:
    model.cons.add(
        sum([
            model.dv_dualproblem[d] *
            DualProblemData.loc[f,d]
            for d in D
        ])
        >= DualProblemData.loc[f,'Profit'])
    
SolverFactory('glpk').solve(model)
model.display()

Model Dual Problem - Solved with Pyomo Method

  Variables:
    dv_dualproblem : Size=3, Index={Fertilizer One, Fertilizer Two, Storage}
        Key            : Lower : Value : Upper : Fixed : Stale : Domain
        Fertilizer One :     0 :   0.0 :  None : False : False : NonNegativeIntegers
        Fertilizer Two :     0 :   0.0 :  None : False : False : NonNegativeIntegers
               Storage :     0 :   1.0 :  None : False : False : NonNegativeIntegers

  Objectives:
    cost : Size=1, Index=None, Active=True
        Key  : Active : Value
        None :   True : 8000.0

  Constraints:
    cons : Size=2
        Key : Lower : Body : Upper
          1 :  2.25 :  3.0 :  None
          2 :   2.6 :  3.0 :  None


### Product Mix Challenge

You manufacture two products, A and B, each of which you sell for $1 profit.  Product A requires 5 blobs and 3 globs, and product B requires 3 blobs and 5 globs.  Your supplier has 120 blobs and 120 globs available.  To maximize profit, how much of each product should you produce? How much profit can you make?

In [16]:
model = ConcreteModel("Product Mix Challenge - Solved Manually") ##the open parenthesis allow us to later name the model by putting the name in single quotes in the parenthesis

#step 1: identify the decision variables 
#DVS
#model.something will make up all
#defining the DVs as variables 
#other options for domain are binary, integer (which means a whole number but also negative or zero), nonnegative reals
model.a = Var(domain=NonNegativeIntegers) #product A
model.b = Var(domain=NonNegativeIntegers) #product B


#step 2: define the objective function
#Obj. Fct.
#Objective() tells python this is the objective function
#expr stands for expression
#sense= allows up to tell python what to do. the only two we use is maximize or minimize
model.profit = Objective(expr = model.a + model.b, sense=maximize)


#step 3: identify the constraints 
#Contraint() tells python this is the objective function
#equality options: <=, >=, ==
model.blobs = Constraint(expr = 5*model.a + 3*model.b <=120) 
model.globs = Constraint(expr = 3*model.a + 5*model.b <=120)

SolverFactory('glpk').solve(model)
model.display()

Model Product Mix Challenge - Solved Manually

  Variables:
    a : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :  15.0 :  None : False : False : NonNegativeIntegers
    b : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :  15.0 :  None : False : False : NonNegativeIntegers

  Objectives:
    profit : Size=1, Index=None, Active=True
        Key  : Active : Value
        None :   True :  30.0

  Constraints:
    blobs : Size=1
        Key  : Lower : Body  : Upper
        None :  None : 120.0 : 120.0
    globs : Size=1
        Key  : Lower : Body  : Upper
        None :  None : 120.0 : 120.0


In [86]:
print(f'''
Profit:    {model.profit()}
A to produce: {model.a()}
B to pproduce: {model.b()}
blobs: {model.blobs()}
globs: {model.globs()}
''')


Profit:    30.0
A to produce: 15.0
B to pproduce: 15.0
blobs: 120.0
globs: 120.0



### Golf Bags

How many of each bag type should you manufacture over the next three months in order to maximize profit?

In [87]:
print(630*60) ##cutting and dyeing minutes 
print(600*60) ##sewing minutes 
print(708*60) ##finishing minutes 
print(135*60) ##inspection and packaging minutes 

37800
36000
42480
8100


In [18]:
model = ConcreteModel("Golf Bags - Solved Manually") ##the open parenthesis allow us to later name the model by putting the name in single quotes in the parenthesis

#step 1: identify the decision variables 
#DVS
#model.something will make up all
#defining the DVs as variables 
#other options for domain are binary, integer (which means a whole number but also negative or zero), nonnegative reals
model.s = Var(domain=NonNegativeIntegers) #model.r is the name for the DV roses 
model.d = Var(domain=NonNegativeIntegers)


#step 2: define the objective function
#Obj. Fct.
#Objective() tells python this is the objective function
#expr stands for expression
#sense= allows up to tell python what to do. the only two we use is maximize or minimize
model.profit = Objective(expr = 10*model.s + 9*model.d, sense=maximize)


#step 3: identify the constraints 
#Contraint() tells python this is the objective function
#equality options: <=, >=, ==
model.cd = Constraint(expr = 42*model.s + 60*model.d <= 37800)
model.sew = Constraint(expr = 30*model.s + 50*model.d <= 36000)
model.f = Constraint(expr = 60*model.s + 40*model.d <= 42480)
model.ip = Constraint(expr = 6*model.s + 15*model.d <= 8100)
SolverFactory('glpk').solve(model)
model.display()

Model Golf Bags - Solved Manually

  Variables:
    s : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 : 540.0 :  None : False : False : NonNegativeIntegers
    d : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 : 252.0 :  None : False : False : NonNegativeIntegers

  Objectives:
    profit : Size=1, Index=None, Active=True
        Key  : Active : Value
        None :   True : 7668.0

  Constraints:
    cd : Size=1
        Key  : Lower : Body    : Upper
        None :  None : 37800.0 : 37800.0
    sew : Size=1
        Key  : Lower : Body    : Upper
        None :  None : 28800.0 : 36000.0
    f : Size=1
        Key  : Lower : Body    : Upper
        None :  None : 42480.0 : 42480.0
    ip : Size=1
        Key  : Lower : Body   : Upper
        None :  None : 7020.0 : 8100.0


### Baseball Gloves

In [92]:
print(900*60) ##cutting and sewing minutes 
print(300*60) ##finishing minutes 
print(100*60) ##packaging and shipping minutes

54000
18000
6000


In [19]:
model = ConcreteModel("Baseball Gloves - Solved Manually") ##the open parenthesis allow us to later name the model by putting the name in single quotes in the parenthesis

#step 1: identify the decision variables 
#DVS
#model.something will make up all
#defining the DVs as variables 
#other options for domain are binary, integer (which means a whole number but also negative or zero), nonnegative reals
model.f = Var(domain=NonNegativeIntegers) #model.r is the name for the DV roses 
model.c = Var(domain=NonNegativeIntegers)


#step 2: define the objective function
#Obj. Fct.
#Objective() tells python this is the objective function
#expr stands for expression
#sense= allows up to tell python what to do. the only two we use is maximize or minimize
model.profit = Objective(expr = 5*model.f + 8*model.c, sense=maximize)


#step 3: identify the constraints 
#Contraint() tells python this is the objective function
#equality options: <=, >=, ==
model.cs = Constraint(expr = 60*model.f + 90*model.c <= 54000)
model.finishing = Constraint(expr = 30*model.f + 20*model.c <= 18000)
model.ps = Constraint(expr = 12.5*model.f + 15*model.c <= 6000)

SolverFactory('glpk').solve(model)
model.display()

Model Baseball Gloves - Solved Manually

  Variables:
    f : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :   0.0 :  None : False : False : NonNegativeIntegers
    c : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 : 400.0 :  None : False : False : NonNegativeIntegers

  Objectives:
    profit : Size=1, Index=None, Active=True
        Key  : Active : Value
        None :   True : 3200.0

  Constraints:
    cs : Size=1
        Key  : Lower : Body    : Upper
        None :  None : 36000.0 : 54000.0
    finishing : Size=1
        Key  : Lower : Body   : Upper
        None :  None : 8000.0 : 18000.0
    ps : Size=1
        Key  : Lower : Body   : Upper
        None :  None : 6000.0 : 6000.0


### Bike Frames

In [94]:
##each bike frame requires 30 yards 
print(.2 * 30) ##required carbon fiber per frame
print(.1 * 30) ##maximum kevlar per frame

6.0
3.0


In [20]:
model = ConcreteModel("Bike Frames - Solved Manually") ##the open parenthesis allow us to later name the model by putting the name in single quotes in the parenthesis

#step 1: identify the decision variables 
#DVS
#model.something will make up all
#defining the DVs as variables 
#other options for domain are binary, integer (which means a whole number but also negative or zero), nonnegative reals
model.p = Var(domain=NonNegativeIntegers) #yards of professional and standard material to have in each frame 
model.s = Var(domain=NonNegativeIntegers)


#step 2: define the objective function
#Obj. Fct.
#Objective() tells python this is the objective function
#expr stands for expression
#sense= allows up to tell python what to do. the only two we use is maximize or minimize
model.cost = Objective(expr = 7.5*model.s + 9*model.p, sense=minimize)


#step 3: identify the constraints 
#Contraint() tells python this is the objective function
#equality options: <=, >=, ==
model.cf = Constraint(expr = 0.1*model.s + 0.3*model.p >=6)
model.k = Constraint(expr = 0.06*model.s + .12*model.p <=3)
model.yards = Constraint(expr = 1*model.p + 1*model.s == 30)

SolverFactory('glpk').solve(model)
model.display()

Model Bike Frames - Solved Manually

  Variables:
    p : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :  15.0 :  None : False : False : NonNegativeIntegers
    s : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :  15.0 :  None : False : False : NonNegativeIntegers

  Objectives:
    cost : Size=1, Index=None, Active=True
        Key  : Active : Value
        None :   True : 247.5

  Constraints:
    cf : Size=1
        Key  : Lower : Body : Upper
        None :   6.0 :  6.0 :  None
    k : Size=1
        Key  : Lower : Body               : Upper
        None :  None : 2.6999999999999997 :   3.0
    yards : Size=1
        Key  : Lower : Body : Upper
        None :  30.0 : 30.0 :  30.0


In [97]:
print(f'''
Cost:    {model.cost()}
Yards of Standard: {model.s()}
Yards of Professional: {model.p()}
Carbon Fiber: {model.cf()}
Total Yards: {model.yards()}
''')


Cost:    225.0
Yards of Standard: 30.0
Yards of Professional: 0.0
Carbon Fiber: 3.0
Total Yards: 30.0



### Investment Portfolios

##### Problem Description:

In [43]:
## Decision Variables: how many dollars to put in growth, income, and money market funds
## Objective Function: maximize yeild
## Constraints: no more than 5% total risk, 10% in G, 10% in I, and 20% in MM, total $ investment is 1000000

In [35]:
IPyield = pd.read_excel("Portfolio3Data.xlsx", sheet_name="IPYield", index_col=0)
IPmaxrisk = pd.read_excel("Portfolio3Data.xlsx", sheet_name="MaxRisk", index_col=0)

In [36]:
IPyield

,Growth,Income,MM
Yield,0.2,0.1,0.06


In [37]:
IPmaxrisk

,Growth,Income,MM,Max
Risk,0.1,0.05,0.01,50000


In [38]:
model = ConcreteModel("Investment Portfolios")

## create set to declare DVs
F = IPyield.keys() #F for fund type

## DECLARE DVs
model.dv_ip = Var(F, domain=NonNegativeReals) ##dv_ip for investment portfolios

## OBJECTIVE FUNCTION
model.profit = Objective(expr = sum([model.dv_ip[f]*IPyield.loc['Yield',f]for f in F]), sense = maximize)

## CONSTRAINTS
model.cons = ConstraintList()

R = MaxRisk.index #R for risk 

for r in R:
    model.cons.add(sum([model.dv_ip[f]*IPmaxrisk.loc[r,f] for f in F]) <= IPmaxrisk.loc[r, 'Max'])

model.cons.add(model.dv_ip['Growth'] >= 100000)
model.cons.add(model.dv_ip['Income'] >= 100000)
model.cons.add(model.dv_ip['MM'] >= 200000)
model.cons.add(model.dv_ip['Growth'] + model.dv_ip['Income'] + model.dv_ip['MM'] == 1000000)

SolverFactory('glpk').solve(model)
model.display()

Model Investment Portfolios

  Variables:
    dv_ip : Size=3, Index={Income, MM, Growth}
        Key    : Lower : Value    : Upper : Fixed : Stale : Domain
        Growth :     0 : 400000.0 :  None : False : False : NonNegativeReals
        Income :     0 : 100000.0 :  None : False : False : NonNegativeReals
            MM :     0 : 500000.0 :  None : False : False : NonNegativeReals

  Objectives:
    profit : Size=1, Index=None, Active=True
        Key  : Active : Value
        None :   True : 120000.0

  Constraints:
    cons : Size=5
        Key : Lower     : Body      : Upper
          1 :      None :   50000.0 :   50000.0
          2 :  100000.0 :  400000.0 :      None
          3 :  100000.0 :  100000.0 :      None
          4 :  200000.0 :  500000.0 :      None
          5 : 1000000.0 : 1000000.0 : 1000000.0


### Project Selection

##### Problem Description:

In [44]:
##DVs: which projects to select (binary)
##Obj fct: Maximize Profit
##ST: can't go over yearly budget. aka: in a given the sum of cost of all projects selected can't exceed yearly budget

In [8]:
ProjectSelectionprofit = pd.read_excel("Portfolio3Data.xlsx", sheet_name="PSProfit", index_col=0)
ProjectSelectiondata = pd.read_excel("Portfolio3Data.xlsx", sheet_name="PSData", index_col=0)

In [9]:
ProjectSelectionprofit

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
Project,,,,,,,,,,,,,,,
Profit,1500,2000,2500,7000,4000,3000,4500,3500,1500,2000,2500,7000,4000,3000,4500


In [10]:
ProjectSelectiondata

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,Funds
Project,,,,,,,,,,,,,,,,
2016,400,500,200,300,450,650,350,300,500,600,150,250,220,170,420,3500
2017,230,270,430,220,500,450,530,700,800,250,500,340,400,300,400,4500
2018,330,260,430,270,400,320,330,260,430,270,400,320,250,300,260,4000
2019,400,500,200,300,450,650,350,300,500,600,150,250,220,170,420,3450
2020,230,270,430,220,500,450,530,700,800,250,500,340,270,300,400,4500


In [15]:
model = ConcreteModel("Project Selection")

## everything has the full name project selection because python was confusing this "ps" for the ps in production scheduling
P_ProjectSelection = ProjectSelectionprofit.keys()

model.dv_ProjectSelection = Var(P_ProjectSelection, domain=Binary)

model.profit = Objective(expr = sum(model.dv_ProjectSelection[p]*ProjectSelectionprofit.loc['Profit',p]for p in P_ProjectSelection), sense = maximize)

I_ProjectSelection=ProjectSelectiondata.index

model.cons = ConstraintList()

for i in I_ProjectSelection:
    model.cons.add(sum(model.dv_ProjectSelection[p]*ProjectSelectiondata.loc[i,p] for p in P_ProjectSelection) <= ProjectSelectiondata.loc[i, 'Funds'])

SolverFactory('glpk').solve(model)
model.display()

Model Project Selection

  Variables:
    dv_ProjectSelection : Size=15, Index={1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15}
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          1 :     0 :   0.0 :     1 : False : False : Binary
          2 :     0 :   0.0 :     1 : False : False : Binary
          3 :     0 :   1.0 :     1 : False : False : Binary
          4 :     0 :   1.0 :     1 : False : False : Binary
          5 :     0 :   1.0 :     1 : False : False : Binary
          6 :     0 :   1.0 :     1 : False : False : Binary
          7 :     0 :   1.0 :     1 : False : False : Binary
          8 :     0 :   1.0 :     1 : False : False : Binary
          9 :     0 :   0.0 :     1 : False : False : Binary
         10 :     0 :   0.0 :     1 : False : False : Binary
         11 :     0 :   0.0 :     1 : False : False : Binary
         12 :     0 :   1.0 :     1 : False : False : Binary
         13 :     0 :   1.0 :     1 : False : False : Binary
         14 :  

C:\Users\katie\AppData\Local\Temp\ipykernel_11460\1625696577.py:8: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  model.profit = Objective(expr = sum(model.dv_ProjectSelection[p]*ProjectSelectionprofit.loc['Profit',p]for p in P_ProjectSelection), sense = maximize)
C:\Users\katie\AppData\Local\Temp\ipykernel_11460\1625696577.py:15: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  model.cons.add(sum(model.dv_ProjectSelection[p]*ProjectSelectiondata.loc[i,p] for p in P_ProjectSelection) <= ProjectSelectiondata.loc[i, 'Funds'])


### Production Scheduling 

##### Problem Description:

In [ ]:
##write each constraint out as a separate line of code each 

In [ ]:
##DVs: How many printer cases to manufacture M-100 and how many to manufacture with M-200.
##Objective: max profit = Revenue ($18/machine) - $50/hour M-100 and $75/hour M-200 - $6 per lb matrl

##STs: 
#M-100 capacity: 25 cases/hr
#M-200 capacity: 40 cases/hr

#M-100 requires 40lbs mtrl/hr
#M-200 requires 50lbs mtrl/hr

#M-100 time <= 15 hours 
#M-200 time <= 10 hours 
#if either machine runs, must run >= 5 hours 

#matrl capacity: 1000lbs

In [45]:
model = ConcreteModel("Production Scheduling")

model.m100 = Var(domain=NonNegativeIntegers) ##number cases to produce with M100
model.m200 = Var(domain=NonNegativeIntegers) ##number cases to produce with M200

model.revenue = Objective(expr = (18*model.m100 + 18*model.m200)-(2*model.m100 + 1.875*model.m200)-(1.6*model.m100*6 + 1.25*model.m200*6), sense=maximize)

##chemical capacity constraint: convert to 
model.chemcapacity = Constraint(expr = 1.6*model.m100 + 1.25*model.m200 <=1000)

##machine running time constraints: convert to max no. cases produced
model.m100maxcases= Constraint(expr = model.m100 <=375)
model.m200maxcases = Constraint(expr = model.m200 <=400)

SolverFactory('glpk').solve(model)
model.display()

Model Production Scheduling

  Variables:
    m100 : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 : 312.0 :  None : False : False : NonNegativeIntegers
    m200 : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 : 400.0 :  None : False : False : NonNegativeIntegers

  Objectives:
    revenue : Size=1, Index=None, Active=True
        Key  : Active : Value
        None :   True : 5446.799999999999

  Constraints:
    chemcapacity : Size=1
        Key  : Lower : Body  : Upper
        None :  None : 999.2 : 1000.0
    m100maxcases : Size=1
        Key  : Lower : Body  : Upper
        None :  None : 312.0 : 375.0
    m200maxcases : Size=1
        Key  : Lower : Body  : Upper
        None :  None : 400.0 : 400.0


### Assembly Line Equipment

##### Problem Description:

In [46]:
##DVs: how many RoboI, RoboII, and RoboIII machines to buy
##Objective: minimize cost = sum(# each machine * respective purchase price)
##ST: must meet demand of casing stampings for each type of casing. each Robo machine can stamp certain # of each casing type 

In [48]:
ALEcost = pd.read_excel("Portfolio3Data.xlsx", sheet_name="ALECost", index_col=0)
ALEdata = pd.read_excel("Portfolio3Data.xlsx", sheet_name="ALEData", index_col=0)

In [49]:
ALEcost

,RoboI,RoboII,RoboIII
Cost,18500,25000,35000


In [50]:
ALEdata

,RoboI,RoboII,RoboIII,Demand
Type 1,100,265,200,3200
Type 2,130,235,160,2500
Type 3,140,170,260,3500
Type 4,210,220,180,3000
Type 5,80,120,220,2500


In [66]:
model = ConcreteModel("Assembly Line Equipment")

##pulling column names of cost table
C_ale = ALEcost.keys()

##declaring decision variables
model.dv_ale = Var(C_ale, domain=NonNegativeIntegers)

##objective function: minimize cost of buying machines
model.cost = Objective(expr = sum([model.dv_ale[c]*ALEcost.loc['Cost',c]for c in C_ale]), sense = minimize)


I_ale=ALEdata.index

model.cons = ConstraintList()

for i in I_ale:
    model.cons.add(sum(model.dv_ale[c]*ALEdata.loc[i,c] for c in C_ale) >= ALEdata.loc[i,'Demand'])

SolverFactory('glpk').solve(model)
model.display()

Model Assembly Line Equipment

  Variables:
    dv_ale : Size=3, Index={RoboII, RoboI, RoboIII}
        Key     : Lower : Value : Upper : Fixed : Stale : Domain
          RoboI :     0 :   6.0 :  None : False : False : NonNegativeIntegers
         RoboII :     0 :   5.0 :  None : False : False : NonNegativeIntegers
        RoboIII :     0 :   7.0 :  None : False : False : NonNegativeIntegers

  Objectives:
    cost : Size=1, Index=None, Active=True
        Key  : Active : Value
        None :   True : 481000.0

  Constraints:
    cons : Size=5
        Key : Lower  : Body   : Upper
          1 : 3200.0 : 3325.0 :  None
          2 : 2500.0 : 3075.0 :  None
          3 : 3500.0 : 3510.0 :  None
          4 : 3000.0 : 3620.0 :  None
          5 : 2500.0 : 2620.0 :  None


### Manufacturing and Distribution

##### Problem Description

In [67]:
##Business Decision: How many locks should be manufactured at each location and where should they be shipped?
##Decision Variables: How many locks to be shipped between each plant-distributor pair? 28 total decision variables 
##Objective: Minimize cost, which is a function of the manufacturing costs and the shipping costs 
##ST: meet at least 80% of distributor demand, don't exceed plant capacity 

In [ ]:
## this is different that network flow because materials can only flow one way through the pipeline

In [3]:
MDprofit = pd.read_excel("Portfolio3Data.xlsx", sheet_name="M&DCosts", index_col=0)
MDcap = pd.read_excel("Portfolio3Data.xlsx", sheet_name="M&DCap", index_col=0)
MDdemand = pd.read_excel("Portfolio3Data.xlsx", sheet_name="M&DDemand", index_col=0)

In [4]:
MDprofit

,Tacoma,San Diego,Dallas,Denver,St. Louis,Tampa,Baltimore
Macon,2.50,2.75,1.75,2.00,2.1,1.80,1.65
Louisville,1.85,1.90,1.50,1.60,1.0,1.90,1.85
Detroit,2.30,2.25,1.85,1.25,1.5,2.25,2.00
Phoenix,1.90,0.90,1.60,1.75,2.0,2.50,2.65


In [5]:
MDcap

,Capacity,Cost/Unit
Macon,18000,35.50
Louisville,15000,37.50
Detroit,25000,39.00
Phoenix,20000,36.25


In [6]:
MDdemand

,Tacoma,San Diego,Dallas,Denver,St. Louis,Tampa,Baltimore
Demand,6800,11600,10800,10080,14400,12000,7200


In [7]:
model = ConcreteModel("Manufacturing and Distribution")

P_md = MDprofit.index
D_md = MDprofit.keys()

##Declare the DVs
model.dv_md = Var(P_md,D_md,domain=NonNegativeIntegers)

##Objective Function

##manufacturing cost
#first add all DVS in a row
#second multiply that value by the cost
#third add up all those values 

#sum(sum([(model.dv_MD[p,d] for d in D) * (cap.loc[p,'Cost/Unit'] for p in P)]))
    
## shipping cost
#sum([(model.dv_MD[p,d]) * (profit.loc[p,d] for p in P for d in D)])

#model.cost = Objective(expr = sum([sum(sum(model.dv_MD[p,d] for d in D) * cap.loc[p,'Cost/Unit'] for p in P)]) + 
                                  #sum([(model.dv_MD[p,d])*(profit.loc[p,d] for p in P for d in D)]), sense = minimize)

model.cost = Objective(expr = 
                       sum([
                           sum(
                               model.dv_md[p,d] * 
                               MDprofit.loc[p,d] 
                               for d in D_md for p in P_md
                           ) + 
                           sum(
                               sum(
                                   model.dv_md[p,d] for d in D_md
                               ) * 
                               MDcap.loc[p, 'Cost/Unit'] for p in P_md
                           )
                       ]), sense = minimize)
                       
## constraints
model.cons = ConstraintList()

# capacity constraint
for p in P_md:
    model.cons.add(sum(model.dv_md[p,d] for d in D_md) <= MDcap.loc[p, 'Capacity'])

#demand constraint
for d in D_md:
    model.cons.add(sum(model.dv_md[p,d] for p in P_md) >= MDdemand.loc['Demand',d]) 
    
## for every city, sum across the plants for the city
## we can only reference the DVs as a matrix
SolverFactory('glpk').solve(model)
model.display()


Model Manufacturing and Distribution

  Variables:
    dv_md : Size=28, Index={Louisville, Detroit, Macon, Phoenix}*{Denver, Tampa, Baltimore, St. Louis, San Diego, Tacoma, Dallas}
        Key                         : Lower : Value   : Upper : Fixed : Stale : Domain
           ('Detroit', 'Baltimore') :     0 :  1200.0 :  None : False : False : NonNegativeIntegers
              ('Detroit', 'Dallas') :     0 :  8600.0 :  None : False : False : NonNegativeIntegers
              ('Detroit', 'Denver') :     0 : 10080.0 :  None : False : False : NonNegativeIntegers
           ('Detroit', 'San Diego') :     0 :     0.0 :  None : False : False : NonNegativeIntegers
           ('Detroit', 'St. Louis') :     0 :     0.0 :  None : False : False : NonNegativeIntegers
              ('Detroit', 'Tacoma') :     0 :     0.0 :  None : False : False : NonNegativeIntegers
               ('Detroit', 'Tampa') :     0 :     0.0 :  None : False : False : NonNegativeIntegers
        ('Louisville', 'Baltimor

C:\Users\katie\AppData\Local\Temp\ipykernel_11460\3745232960.py:25: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  sum([sum(model.dv_md[p,d] * MDprofit.loc[p,d] for d in D_md for p in P_md) + sum(sum(model.dv_md[p,d] for d in D_md) * MDcap.loc[p, 'Cost/Unit'] for p in P_md)]), sense = minimize)
C:\Users\katie\AppData\Local\Temp\ipykernel_11460\3745232960.py:25: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  sum([sum(model.dv_md[p,d] * MDprofit.loc[p,d] for d in D_md for p in P_md) + sum(sum(model.dv_md[p,d] for d in D_md) * MDcap.loc[p, 'Cost/Unit'] for p in P_md)]), sense = minimize)
C:\Users\katie\AppData\Local\Temp\ipykernel_11460\3745232960.py:32: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the 

### Crushing More Rocks

##### Problem Description

In [ ]:
#Decision Variables: tons of fine, medium, and coarse rock to process
#Objective: minimize cost 
#ST: material composition requirements for fine, medium, and coarse rock, max available material type, if we use a setting must produce at least 50 tons 

In [12]:
CMRcost = pd.read_excel("Portfolio3Data.xlsx", sheet_name="CMRCost", index_col=0)
CMRdata = pd.read_excel("Portfolio3Data.xlsx", sheet_name="CMRData", index_col=0)

In [13]:
CMRcost

,Cost
Fine,8
Medium,5
Coarse,3


In [14]:
CMRdata

,Limestone,Chat,Redi-Mix,Rough
Fine,0.50,0.3,0.20,0.0
Medium,0.20,0.4,0.30,0.1
Coarse,0.05,0.2,0.35,0.4
Demand,50.00,60.0,70.00,30.0


In [81]:
## WHAT IF ANALYSIS: trial #1 - run model without minimum of 50 tons constraint

In [15]:
model = ConcreteModel("Crushing More Rocks")

Cost_cmr = CMRcost.index

model.dv_cmr = Var(Cost_cmr, domain=NonNegativeIntegers)

model.cost = Objective(expr = sum([model.dv_cmr[c]*CMRcost.loc[c,'Cost']for c in Cost_cmr]), sense = minimize)

keys_cmr=CMRdata.keys()

model.cons = ConstraintList()

for i in keys_cmr:
    model.cons.add(sum([model.dv_cmr[c]*CMRdata.loc[c,i] for c in Cost_cmr]) >= CMRdata.loc['Demand',i])

SolverFactory('glpk').solve(model)
model.display()

Model Crushing More Rocks

  Variables:
    dv_cmr : Size=3, Index={Fine, Coarse, Medium}
        Key    : Lower : Value : Upper : Fixed : Stale : Domain
        Coarse :     0 : 134.0 :  None : False : False : NonNegativeIntegers
          Fine :     0 :  76.0 :  None : False : False : NonNegativeIntegers
        Medium :     0 :  27.0 :  None : False : False : NonNegativeIntegers

  Objectives:
    cost : Size=1, Index=None, Active=True
        Key  : Active : Value
        None :   True : 1145.0

  Constraints:
    cons : Size=4
        Key : Lower : Body               : Upper
          1 :  50.0 :               50.1 :  None
          2 :  60.0 : 60.400000000000006 :  None
          3 :  70.0 :               70.2 :  None
          4 :  30.0 : 56.300000000000004 :  None


In [82]:
## trial #2 - add constraint
## optimal solution tells us to run 27 tons on medium
## need to add constraint: a minimum of 50 tons of rock must be produced on each setting

In [80]:
model = ConcreteModel("Crushing More Rocks")

Cost_cmr = CMRcost.index

model.dv_cmr = Var(Cost_cmr, domain=NonNegativeIntegers)

model.cost = Objective(expr = sum([model.dv_cmr[c]*CMRcost.loc[c,'Cost']for c in Cost_cmr]), sense = minimize)

keys_cmr=CMRdata.keys()

model.cons = ConstraintList()

for i in keys_cmr:
    model.cons.add(sum([model.dv_cmr[c]*CMRdata.loc[c,i] for c in Cost_cmr]) >= CMRdata.loc['Demand',i])

model.cons.add(model.dv_cmr['Fine'] >= 50) 
model.cons.add(model.dv_cmr['Medium'] >= 50)
model.cons.add(model.dv_cmr['Coarse'] >= 50)

SolverFactory('glpk').solve(model)
model.display()

Model Crushing More Rocks

  Variables:
    dv_cmr : Size=3, Index={Medium, Coarse, Fine}
        Key    : Lower : Value : Upper : Fixed : Stale : Domain
        Coarse :     0 : 118.0 :  None : False : False : NonNegativeIntegers
          Fine :     0 :  68.0 :  None : False : False : NonNegativeIntegers
        Medium :     0 :  51.0 :  None : False : False : NonNegativeIntegers

  Objectives:
    cost : Size=1, Index=None, Active=True
        Key  : Active : Value
        None :   True : 1153.0

  Constraints:
    cons : Size=7
        Key : Lower : Body               : Upper
          1 :  50.0 :               50.1 :  None
          2 :  60.0 :               64.4 :  None
          3 :  70.0 :  70.19999999999999 :  None
          4 :  30.0 : 52.300000000000004 :  None
          5 :  50.0 :               68.0 :  None
          6 :  50.0 :               51.0 :  None
          7 :  50.0 :              118.0 :  None


### Hospital Scheduling

##### Problem Description

In [ ]:
#Decision variables: No. of face lift, lipo, and implant patients to be admitted each week
#Objective function: maximize profit
#490 total bed days available each week, 165 surgical suite hours available per week

In [83]:
HSprofit = pd.read_excel("Portfolio3Data.xlsx", sheet_name="HSProfit", index_col=0)
HSdata = pd.read_excel("Portfolio3Data.xlsx", sheet_name="HSData", index_col=0)

In [84]:
HSprofit

,Face Lift,Lipo,Implants
Profit,240,225,425


In [85]:
HSdata

,Face Lift,Lipo,Implants,Available
Stay,3,5.0,6,490
Surgery Hours,2,1.5,3,165


In [87]:
model = ConcreteModel("Hospital Scheduling")

P_hs = HSprofit.keys()

model.dv_hs = Var(P_hs, domain=NonNegativeIntegers)

model.profit = Objective(expr = sum([model.dv_hs[p]*HSprofit.loc['Profit', p]for p in P_hs]), sense = maximize)

I_hs=HSdata.index

model.cons = ConstraintList()

for i in I_hs:
    model.cons.add(sum([model.dv_hs[p]*HSdata.loc[i,p] for p in P_hs]) <= HSdata.loc[i,'Available'])

SolverFactory('glpk').solve(model)
model.display()

Model Hospital Scheduling

  Variables:
    dv_hs : Size=3, Index={Lipo, Face Lift, Implants}
        Key       : Lower : Value : Upper : Fixed : Stale : Domain
        Face Lift :     0 :   0.0 :  None : False : False : NonNegativeIntegers
         Implants :     0 :  15.0 :  None : False : False : NonNegativeIntegers
             Lipo :     0 :  80.0 :  None : False : False : NonNegativeIntegers

  Objectives:
    profit : Size=1, Index=None, Active=True
        Key  : Active : Value
        None :   True : 24375.0

  Constraints:
    cons : Size=2
        Key : Lower : Body  : Upper
          1 :  None : 490.0 : 490.0
          2 :  None : 165.0 : 165.0


### Butter

##### Problem Description

In [25]:
##Decision Variables: No. of apple butter jars (in 1000s) and peanut butter jars (in 1000s) to produce this week
##Objective: maximize profit
##ST: 40 hours of packaging and 40 hours of sterilization

In [88]:
Bprofit = pd.read_excel("Portfolio3Data.xlsx", sheet_name="BProfit", index_col=0)
Bdata = pd.read_excel("Portfolio3Data.xlsx", sheet_name="BData", index_col=0)

In [89]:
Bprofit

,Peanut,Apple
Profit,1.1,1.3


In [90]:
Bdata

,Peanut,Apple,Capacity
Packaging,0.005,0.004,40
Sterilization,0.004,0.006,40


In [35]:
##WHAT IF ANALYSIS: if apple butter is produced, at least 5000 jars should be produced

In [ ]:
##trial no.1: no min apple butter constraint
##profit: $999.7 and no. apple jars: 2858

In [91]:
model = ConcreteModel("Butter")

P_b = Bprofit.keys()

model.dv_b = Var(P_b, domain=NonNegativeIntegers)

model.profit = Objective(expr = sum([model.dv_b[p]*Bprofit.loc['Profit', p]for p in P_b]), sense = maximize)

I_b=Bdata.index

model.cons = ConstraintList()

for i in I_b:
    model.cons.add(sum([model.dv_b[p]*Bdata.loc[i,p] for p in P_b]) <= Bdata.loc[i,'Capacity'])
    

SolverFactory('glpk').solve(model)
model.display()

Model Butter

  Variables:
    dv_b : Size=2, Index={Apple, Peanut}
        Key    : Lower : Value  : Upper : Fixed : Stale : Domain
         Apple :     0 : 2858.0 :  None : False : False : NonNegativeIntegers
        Peanut :     0 : 5713.0 :  None : False : False : NonNegativeIntegers

  Objectives:
    profit : Size=1, Index=None, Active=True
        Key  : Active : Value
        None :   True : 9999.7

  Constraints:
    cons : Size=2
        Key : Lower : Body   : Upper
          1 :  None : 39.997 :  40.0
          2 :  None :   40.0 :  40.0


In [7]:
##trial no.2: adding the min apple butter constraint
##proft: $9250 and no. apple jars: 5000

In [36]:
profit = pd.read_excel("Portfolio3Data.xlsx", sheet_name="BProfit", index_col=0)
data = pd.read_excel("Portfolio3Data.xlsx", sheet_name="BData", index_col=0)

In [92]:
model = ConcreteModel("Butter")

P_b = Bprofit.keys()

model.dv_b = Var(P_b, domain=NonNegativeIntegers)

model.profit = Objective(expr = sum([model.dv_b[p]*Bprofit.loc['Profit', p]for p in P_b]), sense = maximize)

I_b=Bdata.index

model.cons = ConstraintList()

for i in I_b:
    model.cons.add(sum([model.dv_b[p]*Bdata.loc[i,p] for p in P_b]) <= Bdata.loc[i,'Capacity'])

model.cons.add(model.dv_b['Apple']>=5000)

SolverFactory('glpk').solve(model)
model.display()

Model Butter

  Variables:
    dv_b : Size=2, Index={Apple, Peanut}
        Key    : Lower : Value  : Upper : Fixed : Stale : Domain
         Apple :     0 : 5000.0 :  None : False : False : NonNegativeIntegers
        Peanut :     0 : 2500.0 :  None : False : False : NonNegativeIntegers

  Objectives:
    profit : Size=1, Index=None, Active=True
        Key  : Active : Value
        None :   True : 9250.0

  Constraints:
    cons : Size=3
        Key : Lower  : Body   : Upper
          1 :   None :   32.5 :  40.0
          2 :   None :   40.0 :  40.0
          3 : 5000.0 : 5000.0 :  None


In [11]:
##trial no.3: no apple butter
##proft: $8800 and no. apple jars: 0

In [93]:
model = ConcreteModel("Butter")

P_b = Bprofit.keys()

model.dv_b = Var(P_b, domain=NonNegativeIntegers)

model.profit = Objective(expr = sum([model.dv_b[p]*Bprofit.loc['Profit', p]for p in P_b]), sense = maximize)

I_b=Bdata.index

model.cons = ConstraintList()

for i in I_b:
    model.cons.add(sum([model.dv_b[p]*Bdata.loc[i,p] for p in P_b]) <= Bdata.loc[i,'Capacity'])

model.cons.add(model.dv_b['Apple']==0)

SolverFactory('glpk').solve(model)
model.display()

Model Butter

  Variables:
    dv_b : Size=2, Index={Apple, Peanut}
        Key    : Lower : Value  : Upper : Fixed : Stale : Domain
         Apple :     0 :    0.0 :  None : False : False : NonNegativeIntegers
        Peanut :     0 : 8000.0 :  None : False : False : NonNegativeIntegers

  Objectives:
    profit : Size=1, Index=None, Active=True
        Key  : Active : Value
        None :   True : 8800.0

  Constraints:
    cons : Size=3
        Key : Lower : Body : Upper
          1 :  None : 40.0 :  40.0
          2 :  None : 32.0 :  40.0
          3 :   0.0 :  0.0 :   0.0


In [12]:
### we should include the minimum of 5000 jars apple butter constraint

### Pet Food

##### Problem Description

In [ ]:
## Decision Variables: 
    #tons of seeds in bird food
    #tons of stone in bird food
    #tons of cereal in bird food
    #total tons of bird food manufactured each day**
    #tons of meat in dog food
    #tons of fishmeal in dog food
    #tons of cereal in dog food
    #total tons of dog food manufactured each day**
    
##Objective: maximize project (revenue of $750/ton bird food and $980/ton dog food - cost of seeds, stones,
    #cereal, meat, and fishmeal
    
##ST: 
    #8 hours per day available of blending and packaging each
    #composition requirements that bird and dog food havd for protein, carbs, trace minerals, abrasives, and seeds

In [96]:
PFrevenue = pd.read_excel("PetFood2.xlsx", sheet_name="PFRevenue", index_col=0)
PFcost = pd.read_excel("PetFood2.xlsx", sheet_name="PFCost", index_col=0)
PFmtrl = pd.read_excel("PetFood2.xlsx", sheet_name="PFMtrl", index_col=0)
PFdpt = pd.read_excel("PetFood2.xlsx", sheet_name="PFDpt", index_col=0)

In [97]:
PFrevenue

,Bseeds,Bstones,Bcereal,Dmeat,Dfishmeal,Dcereal
Revenue,750,750,750,980,980,980


In [98]:
PFcost

,Bseeds,Bstones,Bcereal,Dmeat,Dfishmeal,Dcereal
Cost,700,100,200,600,900,200


In [99]:
PFmtrl

,Bseeds,Bstones,Bcereal,Dmeat,Dfishmeal,Dcereal,Brequired,Drequired
Protein,0.10,0.00,0.03,0.12,0.20,0.03,0.05,0.11
Carbs,0.10,0.00,0.30,0.10,0.08,0.30,0.18,0.15
Minerals,0.02,0.03,0.00,0.01,0.02,0.00,0.01,0.01
Abrasives,0.01,1.00,0.00,0.00,0.00,0.00,0.02,0.00
Seeds,1.00,0.00,0.00,0.00,0.00,0.00,0.10,0.00


In [100]:
PFdpt

,Bseeds,Bstones,Bcereal,Dmeat,Dfishmeal,Dcereal,DptMax
Blending,0.25,0.25,0.25,0.15,0.15,0.15,8
Packaging,0.10,0.10,0.10,0.30,0.30,0.30,8


In [101]:
model = ConcreteModel("Pet Food")

##creating set for DVS
I_pf=PFrevenue.keys() #I for ingredients

##DECLARE DVs
model.dv_pf = Var(I_pf, domain=NonNegativeReals)

##OBJECTIVE FUNCTION
#MA: Xprofit  = rev per ton of ingredient - cost per ton of ingredient
model.profit = Objective(expr = 
                         sum([model.dv_pf[i]*PFrevenue.loc['Revenue',i]for i in I_pf]) -
                         sum([model.dv_pf[i]*PFcost.loc['Cost',i] for i in I_pf]),
                         sense = maximize)

##CONSTRAINTS
model.cons = ConstraintList()

#both of our material composition constraints require us to multiply a percentage by the total tons of bird food and dog food produced,
# so we have to calculate the total tons 

TotalBF=model.dv_pf['Bseeds'] + model.dv_pf['Bstones'] + model.dv_pf['Bcereal']
TotalDF=model.dv_pf['Dmeat'] + model.dv_pf['Dfishmeal'] + model.dv_pf['Dcereal']


##bird material composition constraint

M_pf = PFmtrl.index #M for material

for m in M_pf:
    model.cons.add( (model.dv_pf['Bseeds']*PFmtrl.loc[m,'Bseeds'] 
                   + model.dv_pf['Bstones']*PFmtrl.loc[m,'Bstones'] 
                   + model.dv_pf['Bcereal']*PFmtrl.loc[m,'Bcereal']) >= PFmtrl.loc[m,'Brequired'] * TotalBF)

##dog material composition constraint

for m in M_pf:
    model.cons.add( (model.dv_pf['Dmeat']*PFmtrl.loc[m,'Dmeat'] 
                   + model.dv_pf['Dfishmeal']*PFmtrl.loc[m,'Dfishmeal'] 
                   + model.dv_pf['Dcereal']*PFmtrl.loc[m,'Dcereal']) >= PFmtrl.loc[m,'Drequired'] * TotalDF)

##department capacity constraint

D_pf = PFdpt.index #D for department

for d in D_pf:
    model.cons.add(sum([model.dv_pf[i] * PFdpt.loc[d,i] for i in I_pf]) <= PFdpt.loc[d,'DptMax'])
            
SolverFactory('glpk').solve(model)
model.display()

Model Pet Food

  Variables:
    dv_pf : Size=6, Index={Bseeds, Dmeat, Dfishmeal, Bcereal, Bstones, Dcereal}
        Key       : Lower : Value            : Upper : Fixed : Stale : Domain
          Bcereal :     0 : 11.1111111111111 :  None : False : False : NonNegativeReals
           Bseeds :     0 : 6.66666666666667 :  None : False : False : NonNegativeReals
          Bstones :     0 : 2.22222222222222 :  None : False : False : NonNegativeReals
          Dcereal :     0 :             10.0 :  None : False : False : NonNegativeReals
        Dfishmeal :     0 :             10.0 :  None : False : False : NonNegativeReals
            Dmeat :     0 :              0.0 :  None : False : False : NonNegativeReals

  Objectives:
    profit : Size=1, Index=None, Active=True
        Key  : Active : Value
        None :   True : 16488.888888888883

  Constraints:
    cons : Size=12
        Key : Lower : Body                   : Upper
          1 :  None : -4.440892098500626e-16 :   0.0
          2

### Employee Scheduling

##### Problem Description

In [ ]:
## Decision variables: how many employees to hire for shifts 1,2,3,4,5,6,7
## Objective: minimize wage costs 
## Constraints: must meed 99% of demand 

In [102]:
EScost = pd.read_excel("Portfolio3Data.xlsx", sheet_name="ESCost", index_col=0)
ESdemand = pd.read_excel("Portfolio3Data.xlsx", sheet_name="ESDemand", index_col=0)
ESshift = pd.read_excel("Portfolio3Data.xlsx", sheet_name="ESShift", index_col=0)

In [103]:
EScost

,1,2,3,4,5,6,7
Cost,680,705,705,705,705,680,655


In [104]:
ESdemand

,Demand
Sunday,17302.201179
Monday,25631.132594
Tuesday,21101.467706
Wednesday,24919.494949
Thursday,23554.342348
Friday,20388.683176
Saturday,17707.072963


In [105]:
ESshift

,1,2,3,4,5,6,7
Sunday,0,1,1,1,1,1,0
Monday,0,0,1,1,1,1,1
Tuesday,1,0,0,1,1,1,1
Wednesday,1,1,0,0,1,1,1
Thursday,1,1,1,0,0,1,1
Friday,1,1,1,1,0,0,1
Saturday,1,1,1,1,1,0,0


In [106]:
model = ConcreteModel("Employee Scheduling")

C_es = EScost.keys()

##DECLARE DVs
model.dv_es = Var(C_es, domain=NonNegativeIntegers)

##OBJECTIVE FUNCTION
#MIN: cost  = cost per shift per employee (shift is their work for 5 days)

model.cost = Objective(expr = sum([model.dv_es[c]*EScost.loc['Cost',c] for c in C_es]), sense = minimize)


                       
##CONSTRAINTS
model.cons = ConstraintList()

##demand constraint
D_es = ESshift.index

for d in D_es:
    model.cons.add(sum(([model.dv_es[c] * ESshift.loc[d,c] for c in C_es])*1000) >= ESdemand.loc[d,'Demand'])
    
                       
SolverFactory('glpk').solve(model)
model.display()

Model Employee Scheduling

  Variables:
    dv_es : Size=7, Index={1, 2, 3, 4, 5, 6, 7}
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          1 :     0 :   3.0 :  None : False : False : NonNegativeIntegers
          2 :     0 :   2.0 :  None : False : False : NonNegativeIntegers
          3 :     0 :   6.0 :  None : False : False : NonNegativeIntegers
          4 :     0 :   0.0 :  None : False : False : NonNegativeIntegers
          5 :     0 :   7.0 :  None : False : False : NonNegativeIntegers
          6 :     0 :   3.0 :  None : False : False : NonNegativeIntegers
          7 :     0 :  10.0 :  None : False : False : NonNegativeIntegers

  Objectives:
    cost : Size=1, Index=None, Active=True
        Key  : Active : Value
        None :   True : 21205.0

  Constraints:
    cons : Size=7
        Key : Lower              : Body    : Upper
          1 :   17302.2011793957 : 18000.0 :  None
          2 :  25631.13259405359 : 26000.0 :  None
          3 : 21101.467706

### Lockbox Problem

##### Problem Description:

In [ ]:
## Decision Variables: at which region-lockbox locations should masterdebt open lockboxes? there are 42 binary DVs
## Objective Function: MIN cost of interest = sum (float time for given region-lockbox location
                                                    # *ave daily paymebt for that region
                                                    # *.15)
                                                #+ sum (operation cost per location if lockbox is opened at that location)
        
## Constraints: every region needs to be assigned a lockbox

In [56]:
LBinterest = pd.read_excel("Portfolio3Data.xlsx", sheet_name="LBInterestCost", index_col=0)
LBopcost = pd.read_excel("Portfolio3Data.xlsx", sheet_name="LBOpCost", index_col=0)

In [108]:
LBinterest

,Sacramento,Denver,Chicago,Dallas,New York,Atlanta
Central,27.0,13.5,13.50,13.5,20.25,20.25
Mid-Atlantic,58.5,39.0,29.25,39.0,19.50,19.50
Midwest,22.5,15.0,22.50,15.0,37.50,30.00
Northeast,81.0,54.0,27.00,67.5,27.00,40.50
Northwest,21.0,31.5,52.50,42.0,63.00,73.50
Southeast,84.0,48.0,36.00,24.0,48.00,24.00
Southwest,18.0,27.0,54.00,18.0,63.00,54.00


In [110]:
LBopcost

,Sacramento,Denver,Chicago,Dallas,New York,Atlanta
Cost,25,60,35,35,30,35


In [60]:
model = ConcreteModel("Lockbox")

R_lb = LBinterest.index #R for region
C_lb = LBinterest.keys() #C for city
OC_lb = LBopcost.keys() #OC for operating cost

## DECLARE DVS
# declaire the 42 variables 
model.dv_lb = Var(R_lb,C_lb,domain=Binary)

# declare the 6 indicator variables
##when youre reading in two 
model.ind_lb = Var(OC_lb, domain=Binary)

## OBJECTIVE FUNCTION
model.cost = Objective(expr = sum(model.dv_lb[r,c] * LBinterest.loc[r,c] for c in C_lb for r in R_lb) + sum([model.ind_lb[c] * LBopcost.loc['Cost', c] for c in OC_lb]), sense = minimize)


## CONSTRAINTS
model.cons = ConstraintList()

for r in R_lb:
    model.cons.add(sum(model.dv_lb[r,c] for c in C_lb) == 1)

for r in R_lb:
    for c in C_lb:
        model.cons.add(model.ind_lb[c] >= model.dv_lb[r,c])
  ##two outer loops is better coding practice                          
                            
SolverFactory('glpk').solve(model)
model.display()

Model Lockbox

  Variables:
    dv_lb : Size=42, Index={Central, Northwest, Southeast, Mid-Atlantic, Northeast, Southwest, Midwest}*{Chicago, Sacramento, New York , Dallas, Atlanta, Denver}
        Key                            : Lower : Value : Upper : Fixed : Stale : Domain
                ('Central', 'Atlanta') :     0 :   0.0 :     1 : False : False : Binary
                ('Central', 'Chicago') :     0 :   0.0 :     1 : False : False : Binary
                 ('Central', 'Dallas') :     0 :   1.0 :     1 : False : False : Binary
                 ('Central', 'Denver') :     0 :   0.0 :     1 : False : False : Binary
              ('Central', 'New York ') :     0 :   0.0 :     1 : False : False : Binary
             ('Central', 'Sacramento') :     0 :   0.0 :     1 : False : False : Binary
           ('Mid-Atlantic', 'Atlanta') :     0 :   0.0 :     1 : False : False : Binary
           ('Mid-Atlantic', 'Chicago') :     0 :   0.0 :     1 : False : False : Binary
            ('Mid-

C:\Users\katie\AppData\Local\Temp\ipykernel_23288\1979585335.py:16: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  model.cost = Objective(expr = sum(model.dv_lb[r,c] * LBinterest.loc[r,c] for c in C_lb for r in R_lb) + sum([model.ind_lb[c] * LBopcost.loc['Cost', c] for c in OC_lb]), sense = minimize)
C:\Users\katie\AppData\Local\Temp\ipykernel_23288\1979585335.py:23: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  model.cons.add(sum(model.dv_lb[r,c] for c in C_lb) == 1)


### Farming

##### Problem Description:

In [2]:
## Decision Variables: no acres milo on farm 1,2,3. no acres cotton on farm 1,2,3. no acres of wheat on farm 1,2,3
## Objective: max profit
## Constraints: maximum acreage per farm, max water available per farm, harvesting capacity per crop type, required water per crop

In [112]:
FProfit = pd.read_excel("Portfolio3Data.xlsx", sheet_name="FProfit", index_col=0)
FarmSTs = pd.read_excel("Portfolio3Data.xlsx", sheet_name="FarmSTs", index_col=0)
CropSTs = pd.read_excel("Portfolio3Data.xlsx", sheet_name="CropSTs", index_col=0)

In [113]:
FProfit

,Farm 1,Farm 2,Farm 3
Milo,400,400,400
Cotton,300,300,300
Wheat,100,100,100


In [114]:
FarmSTs

,Farm 1,Farm 2,Farm 3
Acres,400,600,300
WaterCap,1500,2000,900


In [115]:
CropSTs

,Capacity,WaterReq
Milo,700,6
Cotton,800,4
Wheat,300,2


In [ ]:
## why is for f in F outside parenthesis from for c in F in objective function

In [116]:
model = ConcreteModel("Farming")

## create sets for DVs
C_f = FProfit.index #C for crop
F_f = FProfit.keys() #F for farm

## DECLARE DVS
model.dv_f = Var(C_f,F_f,domain=NonNegativeReals)

## OBJECTIVE FUNCTION
model.profit = Objective(expr = 
                         sum(
                             model.dv_f[c,f] * 
                             FProfit.loc[c,f] 
                             for c in C_f for f in F_f),
                             sense = maximize)

## CONSTRAINTS
model.cons = ConstraintList()

#each farm has an acreage capacity
for f in F_f:
    model.cons.add(sum([model.dv_f[c,f] for c in C_f]) <= FarmSTs.loc['Acres',f])
                                                     
#each crop has harvesting capacity
for c in C_f:
    model.cons.add(sum([model.dv_f[c,f] for f in F_f]) <= CropSTs.loc[c, 'Capacity'])

#each farm has a water capacity. this is where we used the acres of water required for each crop 
for f in F_f:
    model.cons.add(sum([model.dv_f[c,f] * CropSTs.loc[c,'WaterReq'] for c in C_f]) <= FarmSTs.loc['WaterCap',f])
    
SolverFactory('glpk').solve(model)
model.display()

Model Farming

  Variables:
    dv_f : Size=9, Index={Cotton, Wheat, Milo}*{Farm 1, Farm 2, Farm 3}
        Key                  : Lower : Value : Upper : Fixed : Stale : Domain
        ('Cotton', 'Farm 1') :     0 : 375.0 :  None : False : False : NonNegativeReals
        ('Cotton', 'Farm 2') :     0 : 200.0 :  None : False : False : NonNegativeReals
        ('Cotton', 'Farm 3') :     0 : 225.0 :  None : False : False : NonNegativeReals
          ('Milo', 'Farm 1') :     0 :   0.0 :  None : False : False : NonNegativeReals
          ('Milo', 'Farm 2') :     0 : 200.0 :  None : False : False : NonNegativeReals
          ('Milo', 'Farm 3') :     0 :   0.0 :  None : False : False : NonNegativeReals
         ('Wheat', 'Farm 1') :     0 :   0.0 :  None : False : False : NonNegativeReals
         ('Wheat', 'Farm 2') :     0 :   0.0 :  None : False : False : NonNegativeReals
         ('Wheat', 'Farm 3') :     0 :   0.0 :  None : False : False : NonNegativeReals

  Objectives:
    profit : Si

C:\Users\katie\AppData\Local\Temp\ipykernel_2436\1041810712.py:12: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  sum(


### Inventory Management

##### Problem Description:

In [121]:
IMDVs = pd.read_excel("Portfolio3Data.xlsx", sheet_name="IMDVs", index_col=0)
IMDemand = pd.read_excel("Portfolio3Data.xlsx", sheet_name="IMDemand", index_col=0)

In [122]:
IMDVs

,H,L,LU,LR,HFL,HU,HR
1,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0


In [123]:
IMDemand

,LightDemand,HeavyDemand
1,6,0
2,3,2
3,5,4
4,8,3
5,2,1
6,4,4
7,1,2


In [ ]:
# we will make days of the week an index set
# we wont make light and heavy index set because they dont behave the same in the model

In [59]:
## DVS: #1. no heavy owned H **not day dependent and we do not need to multiply it by 7
        #2. no light owned L **not day dependent
        #3. no light used LU[d] *everything with [d] will change by day
        #4. no light rented LR[d]
        #5. no heavy for light HFL[d]
        #6. no heavy used HU[d] ***not including HFL
        #7. no heavy rent HR[d]
# the trucks we own are constants, they are not day-dependent
# the trucks we rent are day-dependent

## Objective: MIN cost = cost of light owned * no light owned + cost heavy owned * no heavy owned + cost light rented * no light rented + cost heavy rent * no heavy rent
        # sum across our days of: (32L + 44H + 40LU + 54(HU+HFL) + 175LR + 225HR)


## Constraints 

## ST 1: demand per day
# demand changes per day
# the first thing we will do is we will meet heavy demand

#D = IMdays.index
# for every day, Heavy used for each day + heavy rent for each day >= heavy demand for that day 
#for d in D:
 #   HU[d] + HR[d] >= HDemand[d]
    
#for d in D:
 #   HU[d] + HFL[d] <= H
    
## the greatest HU[d] can be is H
## when H <= HD then HU = H
## when HU <= HD then HL = HD-HU
## the obj fct will make sure we go in the right order
## if H >= HD then we rent nothing and HU = 
## HU + HFL <= H
##HFL will only turn on if we need it for the lights but the above calc tells us how much HFL room we have 

##
#for d in D: 

## LU[d] + HFL[d] + LR[d] >= LD[d]
## LU[d] <= L
## when we owned more light than demand, LU = LD, HFL=0, LR =0
## when LU<=D, then LU = L, look for HFL and use that and if we still need more then we will rent lights

In [124]:
model = ConcreteModel("Inventory Management")

D_im=IMDemand.index #D for demand

## DECLARE DVS
model.L = Var(domain=NonNegativeIntegers)
model.H = Var(domain=NonNegativeIntegers)

model.LU = Var(D_im, domain=NonNegativeIntegers)
model.HU = Var(D_im, domain=NonNegativeIntegers)
model.HFL = Var(D_im, domain=NonNegativeIntegers)
model.LR = Var(D_im, domain=NonNegativeIntegers)
model.HR = Var(D_im, domain=NonNegativeIntegers)

## OBJECTIVE FUNCTION
model.cost = Objective(expr = sum([ 
            (32 * model.L)
            + (44 * model.H)
            + (40 * model.LU[d])
            + (54 * (model.HU[d] + model.HFL[d]))
            + (175 * model.LR[d])
            + (225 * model.HR[d])
            for d in D_im]), sense = minimize)

## CONSTRAINTS
model.cons = ConstraintList()

# for every day HU + HR >= Hdemand 
for d in D_im:
    model.cons.add(model.HU[d] + model.HR[d] >= IMDemand.loc[d,'HeavyDemand'])


# for every day HU + HFL <= H
for d in D_im:
    model.cons.add(model.HU[d] + model.HFL[d] <= model.H)
    

# for every day LU + HFL + LR  >= LDemand 
for d in D_im:
    model.cons.add(model.LU[d] + model.HFL[d] + model.LR[d] >= IMDemand.loc[d,'LightDemand'])

# for every day LU <= L
for d in D_im:
    model.cons.add(model.LU[d] <= model.L)
    
    
#for r in R:
 #   model.cons.add(sum(model.dv_pf[f,i] for f in F * PFResources[f,r]) <= PFResources.loc['Capacity',r])
    
SolverFactory('glpk').solve(model)
model.display()

Model Inventory Management

  Variables:
    L : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :   5.0 :  None : False : False : NonNegativeIntegers
    H : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :   4.0 :  None : False : False : NonNegativeIntegers
    LU : Size=7, Index={1, 2, 3, 4, 5, 6, 7}
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          1 :     0 :   5.0 :  None : False : False : NonNegativeIntegers
          2 :     0 :   3.0 :  None : False : False : NonNegativeIntegers
          3 :     0 :   5.0 :  None : False : False : NonNegativeIntegers
          4 :     0 :   5.0 :  None : False : False : NonNegativeIntegers
          5 :     0 :   2.0 :  None : False : False : NonNegativeIntegers
          6 :     0 :   4.0 :  None : False : False : NonNegativeIntegers
          7 :     0 :   1.0 :  None : False : False : NonNegativeIntegers
    HU 

### Inventory Depletion

##### Problem Description:

In [ ]:
## Decision Variables: how many weekender packages and expedition packages to make
## Objective: MAX profit = sum(# each package * the respective cost) - sum(# each package * 1.5)
## Constraints: cant use up more than the current inventories of fruit, meat, and veggies

In [125]:
IDRevenue = pd.read_excel("Portfolio3Data.xlsx", sheet_name="IDRevenue", index_col=0)
IDCost = pd.read_excel("Portfolio3Data.xlsx", sheet_name="IDCost", index_col=0)
IDIngredients = pd.read_excel("Portfolio3Data.xlsx", sheet_name="IDIngredients", index_col=0)

In [126]:
IDRevenue

,Weekender,Expedition
Revenue,3.8,7


In [127]:
IDCost

,Weekender,Expedition
Cost,1.5,1.5


In [128]:
IDIngredients

,Weekender,Expedition,Capacity
Fruit,3,5,10000
Meat,7,18,25000
Vegetables,2,5,12000


In [129]:
model = ConcreteModel("Inventory Depletion")

Package_id = IDRevenue.keys()

model.dv_id = Var(Package_id, domain = NonNegativeIntegers)

model.profit = Objective(expr = 
                         sum([
                             model.dv_id[p] *
                             IDRevenue.loc['Revenue', p] 
                             for p in Package_id]) -
                         sum([
                             model.dv_id[p] *
                             IDCost.loc['Cost',p]
                             for p in Package_id]),
                         sense = maximize)


model.cons = ConstraintList()

I_id = IDIngredients.index #I for ingredients

for i in I_id:
    model.cons.add(
        sum([
            model.dv_id[p] *
            IDIngredients.loc[i,p]
            for p in Package_id
        ]) 
        <= IDIngredients.loc[i,'Capacity'])
    
SolverFactory('glpk').solve(model)
model.display()    

Model Inventory Depletion

  Variables:
    dv_id : Size=2, Index={Expedition, Weekender}
        Key        : Lower : Value  : Upper : Fixed : Stale : Domain
        Expedition :     0 :  263.0 :  None : False : False : NonNegativeIntegers
         Weekender :     0 : 2895.0 :  None : False : False : NonNegativeIntegers

  Objectives:
    profit : Size=1, Index=None, Active=True
        Key  : Active : Value
        None :   True : 8105.0

  Constraints:
    cons : Size=3
        Key : Lower : Body    : Upper
          1 :  None : 10000.0 : 10000.0
          2 :  None : 24999.0 : 25000.0
          3 :  None :  7105.0 : 12000.0


### Second Employee Scheduling

##### Problem Description:

In [130]:
## Decision Variables: how many english and bilingual consultants to start at 7, how many english and biligual consultants to start and 8, etc for 9, 10, and 11
## Objective: minimize wage cost
## Constraints: meet demand

In [43]:
SESCost = pd.read_excel("Portfolio3Data.xlsx", sheet_name="SESCost", index_col=0)
SESMatrix = pd.read_excel("Portfolio3Data.xlsx", sheet_name="SESMatrix", index_col=0)
SESTotalDemand = pd.read_excel("Portfolio3Data.xlsx", sheet_name="SESTDemand", index_col=0)
SESSpanishDemand = pd.read_excel("Portfolio3Data.xlsx", sheet_name="SESSDemand", index_col=0)

In [44]:
SESMatrix

,7,8,9,10,11
7am,1,0,0,0,0
8am,1,1,0,0,0
9am,1,1,1,0,0
10am,1,1,1,1,0
11am,0,1,1,1,1
12pm,1,0,1,1,1
1pm,1,1,0,1,1
2pm,1,1,1,0,1
3pm,1,1,1,1,0
4pm,0,1,1,1,1


In [45]:
SESTotalDemand

,Total_Demand
7am,9
8am,9
9am,9
10am,9
11am,8
12pm,11
1pm,9
2pm,7
3pm,6
4pm,6


In [46]:
SESSpanishDemand

,Spanish_Demand
7am,5
8am,5
9am,4
10am,3
11am,2
12pm,3
1pm,4
2pm,3
3pm,2
4pm,1


In [47]:
SESCost

,7,8,9,10,11
English_Cost,1.0,1.0,1.0,1.0,1.0
Bilingual_Cost,1.1,1.1,1.1,1.1,1.1


In [54]:
model = ConcreteModel("Son of Employee Scheduling")

SES_worker = SESCost.index
SES_shift = SESCost.keys()

model.dv_sonemployeescheduling = Var(SES_worker,SES_shift, domain = NonNegativeIntegers)

model.cost = Objective(expr = 
                       sum([
                           sum(
                               model.dv_sonemployeescheduling[w,s] * 
                               SESCost.loc[w,s]
                               for w in SES_worker for s in SES_shift)
                       ]),
                       sense = minimize)

model.cons = ConstraintList()

T = SESMatrix.index

for t in T:
    model.cons.add(
        sum([
            model.dv_sonemployeescheduling['Bilingual_Cost',s] *
            SESMatrix.loc[t,s] for s in SES_shift
        ])
        >= SESSpanishDemand.loc[t,'Spanish_Demand']
    )

    
for t in T:
    model.cons.add(
        sum([
            sum(model.dv_sonemployeescheduling[w,s] for w in SES_worker) *
            SESMatrix.loc[t,s] for s in SES_shift
        ])
        >= SESTotalDemand.loc[t, 'Total_Demand']
    )

SolverFactory('glpk').solve(model)
model.display()                

Model Son of Employee Scheduling

  Variables:
    dv_sonemployeescheduling : Size=10, Index={English_Cost, Bilingual_Cost}*{7, 8, 9, 10, 11}
        Key                    : Lower : Value : Upper : Fixed : Stale : Domain
         ('Bilingual_Cost', 7) :     0 :   5.0 :  None : False : False : NonNegativeIntegers
         ('Bilingual_Cost', 8) :     0 :   0.0 :  None : False : False : NonNegativeIntegers
         ('Bilingual_Cost', 9) :     0 :   0.0 :  None : False : False : NonNegativeIntegers
        ('Bilingual_Cost', 10) :     0 :   0.0 :  None : False : False : NonNegativeIntegers
        ('Bilingual_Cost', 11) :     0 :   4.0 :  None : False : False : NonNegativeIntegers
           ('English_Cost', 7) :     0 :   4.0 :  None : False : False : NonNegativeIntegers
           ('English_Cost', 8) :     0 :   0.0 :  None : False : False : NonNegativeIntegers
           ('English_Cost', 9) :     0 :   0.0 :  None : False : False : NonNegativeIntegers
          ('English_Cost', 10) :  

C:\Users\katie\AppData\Local\Temp\ipykernel_23288\3636069842.py:10: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  sum(
C:\Users\katie\AppData\Local\Temp\ipykernel_23288\3636069842.py:34: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  sum(model.dv_sonemployeescheduling[w,s] for w in SES_worker) *


### Multiperiod Planning

##### Problem Description:

In [ ]:
## Decision Variables:  #how many pounds of peanuts, walnuts, and almonds to buy and and sell in month 1, 
                        #how many pounds of peanuts, walnuts, and almonds to buy in month 1 and sell in month 2,
                        #how many pounds of peanuts, walnuts, and almonds to buy and sell in month 2
        
## Objective: MAX profit = selling price per lb of mix - cost per lb nut - inventory cost per lb

## Constraints: #chalet no more than 25% peanuts and no less than 40% almonds
                #hovel no more than 60% peanuts and no less than 20% almonds
                #max of 700 lbs mixed each month
                #max availability of each nut each month
                #min mixed chalet of 200 lbs month 2

In [5]:
MPRevenue = pd.read_excel("Portfolio3Data.xlsx", sheet_name="MPRevenue", index_col=0)
MPCost = pd.read_excel("Portfolio3Data.xlsx", sheet_name="MPCost", index_col=0)
MPInvCost = pd.read_excel("Portfolio3Data.xlsx", sheet_name="MPInvCost", index_col=0)
MPAvailability = pd.read_excel("Portfolio3Data.xlsx", sheet_name="MPAvailability", index_col=0)

In [6]:
MPRevenue

,Revenue
CP11,0.80
CW11,0.80
CA11,0.80
HP11,0.40
HW11,0.40
HA11,0.40
CP12,0.81
CW12,0.81
CA12,0.81
HP12,0.39


In [7]:
MPCost

,Cost
CP11,0.20
CW11,0.35
CA11,0.50
HP11,0.20
HW11,0.35
HA11,0.50
CP12,0.20
CW12,0.35
CA12,0.50
HP12,0.20


In [8]:
MPInvCost

,InvCost
CP11,0.00
CW11,0.00
CA11,0.00
HP11,0.00
HW11,0.00
HA11,0.00
CP12,0.02
CW12,0.02
CA12,0.02
HP12,0.02


In [9]:
MPAvailability

,Available
M1Peanuts,400
M1Almonds,200
M2Peanuts,500
M2Almonds,180


In [11]:
model = ConcreteModel("Multi Period Planning")

N_mp = MPRevenue.index #N for nut

## DECLARE DVs
model.dv_mp = Var(N_mp,domain=NonNegativeReals)

## OBJECTIVE FUNCTION
model.profit = Objective(expr = 
                          sum(model.dv_mp[n]*MPRevenue.loc[n,'Revenue']for n in N_mp)    #revenue
                         - sum(model.dv_mp[n]*MPCost.loc[n,'Cost']for n in N_mp)         #sales cost
                         - sum(model.dv_mp[n]*MPInvCost.loc[n,'InvCost']for n in N_mp),  #inventory cost
                          sense = maximize)



## CONSTRAINTS

##PSUEDOCODE
# ST1: chalet and hovel composition reqs for peanuts and almonds: 

# add up lbs of peanuts, almonds, and walnuts for chalet for M11, for hovel for M11, for chalet M12, hovel M12, chalet M22, hovel M22
# multiply those totals by the percentage requirements:
    #M11
    # chalet peanuts M11 <= total p, a, and w for chalet M11 * .25
    # chalet almonds M11 >= total p, a, and w for chalet M11 * .4
    # hovel peanuts M11 <= total p, a, and w for hovel M11 * .6
    # hovel almonds M11 >= total p, a, and w for hovel M11 * .2
    #M12
    # chalet peanuts M12 <= total p, a, and w for chalet M12 * .25
    # chalet almonds M12 >= total p, a, and w for chalet M12 * .4
    # hovel peanuts M12 <= total p, a, and w for hovel M12 * .6
    # hovel almonds M12 >= total p, a, and w for hovel M12 * .2
    #M22
    # chalet peanuts M22 <= total p, a, and w for chalet M22 * .25
    # chalet almonds M22 >= total p, a, and w for chalet M22 * .4
    # hovel peanuts M22 <= total p, a, and w for hovel M22 * .6
    # hovel almonds M22 >= total p, a, and w for hovel M22 * .2
    
    
    
    
#trying to find a way to do a for loop:    
    #peanuts 
    # chalet peanuts M11 <= total p, a, and w for chalet M11 * .25
    # hovel peanuts M11 <= total p, a, and w for hovel M11 * .6
    # chalet peanuts M12 <= total p, a, and w for chalet M12 * .25
    # hovel peanuts M12 <= total p, a, and w for hovel M12 * .6
    # chalet peanuts M22 <= total p, a, and w for chalet M22 * .25
    # hovel peanuts M22 <= total p, a, and w for hovel M22 * .6
    
    #almonds 
    
    
    # code to add up total lbs nuts per mix per period (we will create variables):
    #TotalC11=model.dv_mp['CP11'] + model.dv_mp['CA11'] + model.dv_mp['CW11']
    #TotalH11=model.dv_mp['HP11'] + model.dv_mp['HA11'] + model.dv_mp['HW11']
    #TotalC12=model.dv_mp['CP12'] + model.dv_mp['CA12'] + model.dv_mp['CW12']
    #TotalH12=model.dv_mp['HP12'] + model.dv_mp['HA12'] + model.dv_mp['H12']
    #TotalC22=model.dv_mp['CP22'] + model.dv_mp['CA22'] + model.dv_mp['CW22']
    #TotalH22=model.dv_mp['HP22'] + model.dv_mp['HA22'] + model.dv_mp['HW22']
   # we will plug these variables in on the right hand side of each constraint and multiple the variable by the required percentage 
    
# ST2: pounds of peanuts and almonds available to buy in month 1  and month 2
# CP11 + HP11 + CP12 + HP12 <=400
# CA11 + HA11 + CA12 + HA12 <= 200
# CP22 + HP22 <= 500
# CA22 + HA22 <= 180

# ST3: max of 700 pounds nuts can be mixed per month 
# CP11 + CA11 + CW11 + HP11 + HA11 + HW11 <= 700
# CP12 + CA12 + CW12 + HP12 + HA12 + HW12 + CP22 + CA22 + CW22 + HP22 + HA22 + HW22 <= 700

# ST4: min of 400 pounts of chalet mix in month 2
# CP12 + CA12 + CW12 + CP22 + CA22 + CW22 >= 200



#REAL CODE
# ST1: chalet and hovel composition reqs for peanuts and almonds
model.cons = ConstraintList()

TotalC11=model.dv_mp['CP11'] + model.dv_mp['CA11'] + model.dv_mp['CW11']
TotalH11=model.dv_mp['HP11'] + model.dv_mp['HA11'] + model.dv_mp['HW11']
TotalC12=model.dv_mp['CP12'] + model.dv_mp['CA12'] + model.dv_mp['CW12']
TotalH12=model.dv_mp['HP12'] + model.dv_mp['HA12'] + model.dv_mp['HW12']
TotalC22=model.dv_mp['CP22'] + model.dv_mp['CA22'] + model.dv_mp['CW22']
TotalH22=model.dv_mp['HP22'] + model.dv_mp['HA22'] + model.dv_mp['HW22']

model.cons.add(model.dv_mp['CP11'] <= TotalC11 * 0.25)
model.cons.add(model.dv_mp['HP11'] <= TotalH11 * 0.6)
model.cons.add(model.dv_mp['CP12'] <= TotalC12 * 0.25)
model.cons.add(model.dv_mp['HP12'] <= TotalH12 * 0.6)
model.cons.add(model.dv_mp['CP22'] <= TotalC22 * 0.25)
model.cons.add(model.dv_mp['HP22'] <= TotalH22 * 0.6)

model.cons.add(model.dv_mp['CA11'] >= TotalC11 * 0.4)
model.cons.add(model.dv_mp['HA11'] >= TotalH11 * 0.2)
model.cons.add(model.dv_mp['CA12'] >= TotalC12 * 0.4)
model.cons.add(model.dv_mp['HA12'] >= TotalH12 * 0.2)
model.cons.add(model.dv_mp['CA22'] >= TotalC22 * 0.4)
model.cons.add(model.dv_mp['HA22'] >= TotalH22 * 0.2)


# ST2: available peanuts and almonds to buy in M1 and M2
M1Peanuts=model.dv_mp['CP11'] + model.dv_mp['HP11'] + model.dv_mp['CP12'] + model.dv_mp['HP12']
M1Almonds=model.dv_mp['CA11'] + model.dv_mp['HA11'] + model.dv_mp['CA12'] + model.dv_mp['HA12']
M2Peanuts=model.dv_mp['CP22'] + model.dv_mp['HP22']
M2Almonds=model.dv_mp['CA22'] + model.dv_mp['HA22']


model.cons.add(M1Peanuts <= 400)
model.cons.add(M1Almonds <= 200)
model.cons.add(M2Peanuts <= 500)
model.cons.add(M2Almonds <= 180)


# ST3: max of 700 pounds nuts can be mixed per month 
M1Mix = model.dv_mp['CP11'] + model.dv_mp['CA11'] + model.dv_mp['CW11'] + model.dv_mp['HP11'] + model.dv_mp['HA11'] + model.dv_mp['HW11']
M2Mix = model.dv_mp['CP12'] + model.dv_mp['CA12'] + model.dv_mp['CW12'] + model.dv_mp['HP12'] + model.dv_mp['HA12'] + model.dv_mp['HW12'] + model.dv_mp['CP22'] + model.dv_mp['CA22'] + model.dv_mp['CW22'] + model.dv_mp['HP22'] + model.dv_mp['HA22'] + model.dv_mp['HW22']

model.cons.add(M1Mix <= 700)
model.cons.add(M2Mix <= 700)


# ST4: min of 200 pounts of chalet mix in month 2
M2Chalet = model.dv_mp['CP12'] + model.dv_mp['CA12'] + model.dv_mp['CW12'] + model.dv_mp['CP22'] + model.dv_mp['CA22'] + model.dv_mp['CW22']

model.cons.add(M2Chalet >= 200)


SolverFactory('glpk').solve(model)
model.display()


Model Multi Period Planning

  Variables:
    dv_mp : Size=18, Index={HW11, HW12, HP11, HA11, CP11, CW11, CP22, HA12, CW22, CP12, HA22, HP22, HW22, HP12, CA11, CA22, CW12, CA12}
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        CA11 :     0 : 200.0 :  None : False : False : NonNegativeReals
        CA12 :     0 :   0.0 :  None : False : False : NonNegativeReals
        CA22 :     0 : 180.0 :  None : False : False : NonNegativeReals
        CP11 :     0 : 125.0 :  None : False : False : NonNegativeReals
        CP12 :     0 :   0.0 :  None : False : False : NonNegativeReals
        CP22 :     0 : 112.5 :  None : False : False : NonNegativeReals
        CW11 :     0 : 175.0 :  None : False : False : NonNegativeReals
        CW12 :     0 :   0.0 :  None : False : False : NonNegativeReals
        CW22 :     0 : 157.5 :  None : False : False : NonNegativeReals
        HA11 :     0 :   0.0 :  None : False : False : NonNegativeReals
        HA12 :     0 :   0.0 :  None : F

C:\Users\katie\AppData\Local\Temp\ipykernel_1644\1970770076.py:10: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  sum(model.dv_mp[n]*MPRevenue.loc[n,'Revenue']for n in N_mp)    #revenue
C:\Users\katie\AppData\Local\Temp\ipykernel_1644\1970770076.py:11: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  - sum(model.dv_mp[n]*MPCost.loc[n,'Cost']for n in N_mp)         #sales cost
C:\Users\katie\AppData\Local\Temp\ipykernel_1644\1970770076.py:12: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  - sum(model.dv_mp[n]*MPInvCost.loc[n,'InvCost']for n in N_mp),  #inventory cost


### Snow Removal

##### Problem Description:

In [ ]:
## Decision Variables: which sites should each zone dump its snow off at? aka: 
                        #for a given zone-site pair, is snow being dumped?(50 total binary DVs)
## Objective: MIN cost = amount of snow to move * distance * cost per meter per km
## Constraints: move all snow expected at each zone, don't exceed site capacity

In [140]:
SRCost = pd.read_excel("Portfolio3Data.xlsx", sheet_name="SRCost", index_col=0)
SRAmount = pd.read_excel("Portfolio3Data.xlsx", sheet_name="SRAmount", index_col=0)
SRCapacity = pd.read_excel("Portfolio3Data.xlsx", sheet_name="SRCapacity", index_col=0)

In [141]:
SRCost

,A,B,C,D,E
1,52.02,21.42,74.97,113.22,142.29
2,36.48,31.92,126.16,138.32,133.76
3,21.56,44.66,56.98,144.76,132.44
4,35.88,49.68,62.10,113.16,122.82
5,19.05,39.37,26.67,100.33,111.76
6,54.18,63.21,83.85,99.33,78.69
7,53.28,68.82,109.89,68.82,63.27
8,59.40,66.00,57.20,83.60,53.90
9,40.30,53.30,85.80,97.50,93.60
10,43.20,87.75,95.85,81.00,112.05


In [142]:
SRAmount

,Amount
1,153
2,152
3,154
4,138
5,127
6,129
7,111
8,110
9,130
10,135


In [143]:
SRCapacity

,A,B,C,D,E
Capacity,350,250,500,400,200


In [144]:
model = ConcreteModel("Snow Removal")

Z_sr = SRCost.index #Z for zone
S_sr = SRCost.keys() #S for site

## DECLARE DVs
model.dv_sr = Var(Z_sr,S_sr,domain=Binary)

## OBJECTIVE FUNCTION
model.cost = Objective(expr = 
                         sum(sum([model.dv_sr[z,s] * SRCost.loc[z,s] for s in S_sr]) for z in Z_sr),
                             sense = minimize)


## CONSTRAINTS

model.cons = ConstraintList()
# each zone must be assigned to only one site
for z in Z_sr:
    model.cons.add(sum([model.dv_sr[z,s] for s in S_sr]) == 1)
    
# site capacity cant be exceeded
for s in S_sr:
    model.cons.add(sum([model.dv_sr[z,s] * SRAmount.loc[z,'Amount'] for z in Z_sr]) <= SRCapacity.loc['Capacity',s])

SolverFactory('glpk').solve(model)
model.display()    

Model Snow Removal

  Variables:
    dv_sr : Size=50, Index={1, 2, 3, 4, 5, 6, 7, 8, 9, 10}*{A, D, B, C, E}
        Key       : Lower : Value : Upper : Fixed : Stale : Domain
         (1, 'A') :     0 :   0.0 :     1 : False : False : Binary
         (1, 'B') :     0 :   1.0 :     1 : False : False : Binary
         (1, 'C') :     0 :   0.0 :     1 : False : False : Binary
         (1, 'D') :     0 :   0.0 :     1 : False : False : Binary
         (1, 'E') :     0 :   0.0 :     1 : False : False : Binary
         (2, 'A') :     0 :   1.0 :     1 : False : False : Binary
         (2, 'B') :     0 :   0.0 :     1 : False : False : Binary
         (2, 'C') :     0 :   0.0 :     1 : False : False : Binary
         (2, 'D') :     0 :   0.0 :     1 : False : False : Binary
         (2, 'E') :     0 :   0.0 :     1 : False : False : Binary
         (3, 'A') :     0 :   0.0 :     1 : False : False : Binary
         (3, 'B') :     0 :   0.0 :     1 : False : False : Binary
         (3, 'C') :  

C:\Users\katie\AppData\Local\Temp\ipykernel_2436\1031922586.py:11: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  sum(sum([model.dv_sr[z,s] * SRCost.loc[z,s] for s in S_sr]) for z in Z_sr),


### Network Flow

##### Problem Description:

In [ ]:
## Decision Variables: for every to-from station pair, how much LNG is transported
## Objective: MAX profit achieved by transporting gas = # (revenue of cost willing to transport gas * net in) 
                                                        # - matrix of all LNG transported * matrix of transportation costs 

## Constraints: # pipeline capacities 
                # net in <= demand for each station
                # net out <= supply for each station                

In [3]:
NetworkFlow_Cost = pd.read_excel("NetworkFlow.xlsx", sheet_name="NetworkFlow_Cost", index_col=0)
NetworkFlow_Capacity = pd.read_excel("NetworkFlow.xlsx", sheet_name="NetworkFlow_Capacity", index_col=0)
NetworkFlow_Demand = pd.read_excel("NetworkFlow.xlsx", sheet_name="NetworkFlow_Demand", index_col=0)
NetworkFlow_Supply = pd.read_excel("NetworkFlow.xlsx", sheet_name="NetworkFlow_Supply", index_col=0)
NetworkFlow_AmtWillingtoPay = pd.read_excel("NetworkFlow.xlsx", sheet_name="NetworkFlow_AmtWillingtoPay", index_col=0)

In [4]:
model = ConcreteModel("Network Flow")

I_out = NetworkFlow_Cost.index
J_in = NetworkFlow_Cost.keys()

model.dv_nf = Var(I_out,J_in,domain=NonNegativeReals)

model.profit = Objective(expr = 
                         sum(
                             NetworkFlow_AmtWillingtoPay.loc['Willing_to_Pay',j] *
                             (model.dv_nf[j,i] - model.dv_nf[i,j]) 
                             for i in I_out for j in J_in 
                         ) - 
                         sum(
                             model.dv_nf[i,j] *
                             NetworkFlow_Cost.loc[i,j] 
                             for j in J_in for i in I_out
                         ), 
                         sense = maximize)

model.cons = ConstraintList()

for j in J_in:
    model.cons.add(sum(model.dv_nf[j,i] - model.dv_nf[i,j] for i in I_out) <= NetworkFlow_Demand.loc['Demand',j])
    
for j in J_in:
    model.cons.add(sum(model.dv_nf[i,j] - model.dv_nf[j,i] for i in I_out) <= NetworkFlow_Supply.loc['Supply',j])
    
for j in J_in:
    for i in I_out:
        model.cons.add(model.dv_nf[i,j] <= NetworkFlow_Capacity.loc[i,j])
        
SolverFactory('glpk').solve(model)
model.display()

Model Network Flow

  Variables:
    dv_nf : Size=121, Index={Kiowa, Lebanon, Wharton, Perryville, Katy, Waha, Carthage, Joliet, Henry, Leidy, Maumee}*{Kiowa, Lebanon, Wharton, Perryville, Katy, Waha, Carthage, Joliet, Henry, Leidy, Maumee}
        Key                          : Lower : Value : Upper : Fixed : Stale : Domain
            ('Carthage', 'Carthage') :     0 :   0.0 :  None : False : False : NonNegativeReals
               ('Carthage', 'Henry') :     0 :   0.0 :  None : False : False : NonNegativeReals
              ('Carthage', 'Joliet') :     0 :   0.0 :  None : False : False : NonNegativeReals
                ('Carthage', 'Katy') :     0 :  30.0 :  None : False : False : NonNegativeReals
               ('Carthage', 'Kiowa') :     0 :  20.0 :  None : False : False : NonNegativeReals
             ('Carthage', 'Lebanon') :     0 :   0.0 :  None : False : False : NonNegativeReals
               ('Carthage', 'Leidy') :     0 :   0.0 :  None : False : False : NonNegativeReals
 

C:\Users\katie\AppData\Local\Temp\ipykernel_1644\4072220515.py:9: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  sum(
C:\Users\katie\AppData\Local\Temp\ipykernel_1644\4072220515.py:14: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  sum(
C:\Users\katie\AppData\Local\Temp\ipykernel_1644\4072220515.py:24: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  model.cons.add(sum(model.dv_nf[j,i] - model.dv_nf[i,j] for i in I_out) <= NetworkFlow_Demand.loc['Demand',j])
C:\Users\katie\AppData\Local\Temp\ipykernel_1644\4072220515.py:27: DeprecationWarning: Calling np.sum(generator) is deprecated, and in th